In [ ]:
import random
import warnings
warnings.filterwarnings('ignore')
import numpy.linalg as linalg
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import datetime
from datetime import timedelta, datetime, timezone
import tqdm
import geopandas as gpd
from shapely.geometry import Point
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LightSource
from sklearn.preprocessing import QuantileTransformer
import skgstat as skg
from skgstat import models
import sklearn as sklearn
from sklearn.neighbors import KDTree
import math
from scipy.spatial import distance_matrix
from scipy.interpolate import Rbf
from tqdm import tqdm
from sklearn.metrics import pairwise_distances
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import matplotlib.tri as tri
#import argparse
import shapely
import glob
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
import argparse
import seaborn as sns
from sklearn.linear_model import LinearRegression
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import math
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import FuncFormatter
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pickle

# define paths
root_path = '/scratch/users/pburdeau/data/gas_composition' # root_path to project
ghgrp_path = os.sep.join([root_path, 'ghgrp', 'ghgrp_v2.csv']) # path to ghgrp data
ghgrp = pd.read_csv(ghgrp_path)
shapefiles_path = root_path + '/shapefiles'
usgs_path = root_path + '/usgs/usgs.csv'
out_path = root_path + '/out'

# Clean USGS

# Download geographic data: shale plays, counties, states, basins
us_states = gpd.read_file(shapefiles_path + '/cb_2018_us_state_20m/cb_2018_us_state_20m.shp')

# Clean us_counties to keep only the ones in usgs
us_counties = gpd.read_file(shapefiles_path + '/cb_2018_us_county_500k/cb_2018_us_county_500k.shp')
basins_gdf = gpd.read_file(shapefiles_path + '/basins_shapefiles')


index_to_deletes = []
for i in range(len(us_counties)):
    county_name = us_counties['NAME'].iloc[i].upper()
    state_fp = us_counties['STATEFP'].iloc[i]
    if len(us_states[us_states.STATEFP == state_fp]) > 0:
        state_name = us_states[us_states.STATEFP == state_fp].reset_index(drop=True).NAME.iloc[0].upper()
        combined_name = f"{county_name} ({state_name})"
        us_counties['NAME'].iloc[i] = combined_name
        us_counties['COUNTY_NAME'] = county_name
        us_counties['STATE_NAME'] = state_name
    else:
        index_to_deletes.append(i)
        
for i in index_to_deletes:
    us_counties = us_counties.drop(i)


usgs = pd.read_csv(usgs_path)
usgs_not_na = usgs[~pd.isna(usgs.LAT)]
usgs_not_na = usgs_not_na[~pd.isna(usgs.LONG)]
usgs_not_na = usgs_not_na[~pd.isna(usgs.FINAL_SAMPLING_DATE)]

years = []
for i in range(len(usgs_not_na)):
    years.append(int(usgs_not_na.FINAL_SAMPLING_DATE.iloc[i][-4:]))
usgs_not_na['Year'] = years

usgs_gdf = gpd.GeoDataFrame(usgs_not_na, geometry=gpd.points_from_xy(usgs_not_na.LONG, usgs_not_na.LAT), crs="EPSG:4326")

usgs_gdf = usgs_gdf.to_crs(26914)

usgs_gdf = usgs_gdf.rename(columns={'API':'API14'})

usgs_gdf['COUNTY'] = [str(usgs_gdf.COUNTY.iloc[i]) + ' (' + usgs_gdf.STATE.iloc[i] + ')' for i in range(len(usgs_gdf))]

# transform dates in datetime format
dates = []
with tqdm(total=len(usgs_gdf)) as pbar:
    for i in range(len(usgs_gdf)):
        month, day, year = usgs_gdf['FINAL_SAMPLING_DATE'].iloc[i].split('/')
        month = int(month)
        day = int(day)
        year = int(year)
        if month == 0:
            month = 1
        if day == 0:
            day = 1
        if (month == 2) and (day > 28):
            day = 28
        if (day > 30):
            day = 30
        if month > 12:
            month = 12
        dates.append(datetime(year, month, day, 0, 0, 0))
        pbar.update(1)
    usgs_gdf['date'] = dates    

# Transform datetime in number of days since earliest date
usgs_gdf = usgs_gdf.sort_values(by='date').reset_index(drop=True)
epochs = [0]
for i in range(1, len(usgs_gdf)):
    epochs.append((usgs_gdf.date.iloc[i] - usgs_gdf.date.iloc[0]).days)
usgs_gdf['epochs'] = epochs

usgs_gdf = usgs_gdf.drop(columns=['ID', 'SOURCE','BLM ID','USGS ID','ALASKA_QUAD_NAMES','WELL NAME','PUBLICATION DATE','COUNTY','STATE'])

usgs_gdf = gpd.sjoin(usgs_gdf.to_crs(26914), basins_gdf.to_crs(26914), op='within')

usgs_gdf = usgs_gdf.to_crs(26914)

usgs_gdf['X'] = usgs_gdf.geometry.x
usgs_gdf['Y'] = usgs_gdf.geometry.y
usgs_gdf['T'] = np.array(usgs_gdf.epochs)

usgs_gdf = usgs_gdf.drop(columns=['FINAL_DRILLING_DATE', 'FINAL_SAMPLING_DATE', 'SAMPLE DATE BEFORE COMPLETION', 'index_right', 'BASIN_CODE', 'DEPTH','AGE','FORMATION','geometry','FIELD','LAT','LONG'])

numeric_columns = ['HE', 'CO2', 'H2', 'N2', 'H2S', 'AR', 'O2', 'C1', 'C2', 'C3', 'N-C4', 'I-C4', 'N-C5', 'I-C5', 'C6+']

usgs_gdf[numeric_columns] = usgs_gdf[numeric_columns].apply(pd.to_numeric, errors='coerce')


# Ensure 'geometry' column exists
usgs_gdf['geometry'] = gpd.points_from_xy(usgs_gdf['X'], usgs_gdf['Y'])

# Convert to GeoDataFrame
usgs_gdf = gpd.GeoDataFrame(usgs_gdf, geometry='geometry', crs="EPSG:26914")

# Keep API that are NaNs
key_columns = ['X', 'Y', 'Year', 'date', 'T', 'BASIN_NAME']
aggregated_df = usgs_gdf.groupby(key_columns).agg({col: np.nanmean for col in numeric_columns}).reset_index()
aggregated_df.to_csv(root_path + '/usgs/usgs_processed_with_nanapis.csv')

# Upload production data

wells_data = []
basin_list = usgs_gdf.BASIN_NAME.unique()
with tqdm(total=len(basin_list)) as pbar:
    for basin_name in basin_list:
        basin_file_path = os.path.join(root_path, f'wells_info_prod_per_basin/{basin_name}_final.csv')
        if os.path.isfile(basin_file_path):
            basin_df = pd.read_csv(basin_file_path)
            wells_data.append(basin_df)
        pbar.update(1)

# Merge usgs data that has API with production data

usgs_prod = pd.merge(wells_data[['API14', 'Year', 'Monthly Gas', 'Monthly Oil', 'BASIN_NAME', 'X', 'Y', 'T']], usgs_gdf[['API14', 'HE', 'CO2', 'H2', 'N2', 'H2S', 'AR', 'O2', 'C1', 'C2', 'C3',
       'N-C4', 'I-C4', 'N-C5', 'I-C5', 'C6+', 'Year', 'X', 'Y', 'T']], on=['Year', 'API14'])
usgs_prod = usgs_prod.rename(columns={'X_x':'X_prod', 'Y_x':'Y_prod', 'T_x':'T_prod',
                                     'X_y':'X', 'Y_y':'Y', 'T_y':'T'})
group_cols = ['API14', 'Year', 'X_prod', 'Y_prod', 'T_prod', 'BASIN_NAME']
mean_cols = ['Monthly Gas', 'Monthly Oil', 'HE', 'CO2', 'H2', 'N2', 'H2S',
             'AR', 'O2', 'C1', 'C2', 'C3', 'N-C4', 'I-C4', 'N-C5', 'I-C5', 'C6+']

df_grouped = usgs_prod.groupby(group_cols, dropna=False)[mean_cols].mean().reset_index()
df_grouped.to_csv(root_path + '/usgs/usgs_prod.csv', index=False)

# Correlation analysis

import seaborn as sns
from scipy.stats import linregress

# Load dataset
df = usgs_prod.copy()

# Compute Gas-to-Oil Ratio (GOR) while handling NaN values
df["GOR"] = df["Monthly Gas"] / df["Monthly Oil"]
df["Log GOR"] = np.log(df["GOR"])

# Handle NaN and infinite values
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(subset=["Log GOR"], inplace=True)
lower, upper = np.percentile(df["Log GOR"].dropna(), [0, 100])
df_filtered = df[(df["Log GOR"] >= lower) & (df["Log GOR"] <= upper)].copy()

# Define all components
components = ['C1', 'C2', 'C3', 'N-C4', 'I-C4', 'N-C5', 'I-C5', 'C6+']

# Define a color palette
palette = sns.color_palette("crest", n_colors=len(components))[::-1]  # Shades of blue

# Set up subplots without shared axes
n_rows, n_cols = 2, 4  # Adjust grid to fit all components
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 6))
axes = axes.flatten()  # Flatten axes for easy iteration

for i, (comp, color) in enumerate(zip(components, palette)):
    ax = axes[i]
    valid_mask = df_filtered["Log GOR"].notna() & df_filtered[comp].notna()
    
    if valid_mask.sum() > 1:
        ax.scatter(df_filtered.loc[valid_mask, "Log GOR"], 
                   df_filtered.loc[valid_mask, comp], 
                   alpha=0.3, color=color, edgecolors="none")

        slope, intercept, r_value, _, _ = linregress(df_filtered.loc[valid_mask, "Log GOR"], df_filtered.loc[valid_mask, comp])
        x_vals = np.linspace(df_filtered["Log GOR"].min(), df_filtered["Log GOR"].max(), 100)
        ax.plot(x_vals, slope * x_vals + intercept, color="black")
        ax.text(0.05, 0.9, f"r={r_value:.3f}", transform=ax.transAxes, bbox=dict(facecolor="white", alpha=0.5))
    
    ax.set_title(comp, fontsize=10, fontweight="bold")
    ax.set_xlabel("Log GOR")
    ax.set_ylabel(f"% {comp}")

plt.tight_layout()

plt.savefig('figures_out/correlation_plots.png', dpi=300)
# Adjust layout to prevent overlap
plt.show()

components = ['HE', 'CO2', 'H2', 'N2', 'H2S', 'AR', 'O2','C1', 'C2', 'C3', 'N-C4', 'I-C4', 'N-C5', 'I-C5', 'C6+']
for comp in components:
    valid_mask = df_filtered["Log GOR"].notna() & df_filtered[comp].notna()
    slope, intercept, r_value, _, _ = linregress(df_filtered.loc[valid_mask, "Log GOR"], df_filtered.loc[valid_mask, comp])
    print(f'Comp: {comp}')
    print(f"r={r_value:.3f}")

# Add 'T' info to production files

input_folder = root_path + '/wells_prod_jan_26' # path for production data without 'T' column
output_folder = root_path + '/wells_info_prod_per_basin'

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# List existing files in the output folder
existing_files = set(os.listdir(output_folder))

# Iterate over all CSV files in the input folder
with tqdm(total=len(os.listdir(input_folder))) as pbar:
    for file in os.listdir(input_folder):
        if file.endswith('.csv') and 'Anadarko' in file:
            print(file)
            # Check if the file already exists in the output folder
#             if file in existing_files:
#                 print(f"File '{file}' already processed. Skipping...")
#                 pbar.update(1)
#                 continue  # Skip this file

            file_path = os.path.join(input_folder, file)

            # Read the CSV file
            df = pd.read_csv(file_path)

            # Add geometry and convert to UTM Zone 14N
            df['geometry'] = df.apply(
                lambda row: Point(row['Surface Hole Longitude (WGS84)'], row['Surface Hole Latitude (WGS84)']),
                axis=1
            )
            gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")
            del df
            gdf = gdf.to_crs(26914)  # Convert to UTM Zone 14N
            gdf['X'] = gdf.geometry.x
            gdf['Y'] = gdf.geometry.y
            gdf = gdf.drop(columns='geometry')

            # Convert 'Monthly Production Date' to datetime and extract the year
            gdf['Monthly Production Date'] = pd.to_datetime(gdf['Monthly Production Date'])
            gdf['Year'] = gdf['Monthly Production Date'].dt.year

            # Group by API14, X, Y, and Year, aggregating Monthly Gas and Monthly Oil
            grouped = gdf.groupby(['API14', 'X', 'Y', 'Year'], as_index=False).agg({
                'Monthly Gas': 'sum',
                'Monthly Oil': 'sum',
                'BASIN_NAME': 'first'  # Retain the first value of BASIN_NAME
            })

            # Compute the middle of the year for each group
            grouped['Last Date'] = grouped['Year'].apply(lambda year: pd.Timestamp(f'{year}-12-31'))  # End of the year

            # Calculate T based on the middle date
            grouped['T'] = (grouped['Last Date'] - pd.Timestamp(reference_date)).dt.days

            # Drop the temporary 'Middle Date' column
            grouped = grouped.drop(columns=['Last Date'])

            # Save the grouped data to the output directory
            output_file_path = os.path.join(output_folder, file)
            grouped.to_csv(output_file_path, index=False)
            print(f"Saved grouped data to: {output_file_path}")

        pbar.update(1)


# Clean ghgrp data

ghgrp_indsegm_not_na = ghgrp[ghgrp.industry_segment.notna()]
ghgrp_production = ghgrp_indsegm_not_na[ghgrp_indsegm_not_na.industry_segment.str.contains('Onshore petroleum and natural gas production')]
ghgrp_production = ghgrp_production[ghgrp_production.ch4_average_mole_fraction > 0].reset_index(drop=True)
ghgrp_production = ghgrp_production[ghgrp_production.ch4_average_mole_fraction > ghgrp_production.co2_average_mole_fraction].reset_index(drop=True)

state_abbreviations_to_names = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming'
}


# Sample data

# Extract 'County' and 'State' information (as shown in the previous response)
ghgrp_production['County'] = ghgrp_production['sub_basin_county'].str.extract(r'([A-Z\s]+),\s[A-Z]+\s\(\d+\)')
ghgrp_production['State_abbr'] = ghgrp_production['sub_basin_county'].str.extract(r'([A-Z]+)\s\(\d+\)')

# Add a new column 'State' with the full state names
ghgrp_production['State'] = ghgrp_production['State_abbr'].map(state_abbreviations_to_names).str.upper()

# Add production data to GHGRP

GHGRPfolder = os.sep.join([root_path, 'ghgrp'])
wells_grouped_by_year_folder = root_path + '/wells_info_prod_per_basin'  # Latest data folder

# Collect all files from the EF_W_ONSHORE_WELLS folder
wells_folder = os.path.join(GHGRPfolder, 'EF_W_ONSHORE_WELLS')
all_files = [os.path.join(wells_folder, file) for file in os.listdir(wells_folder) if file.endswith('.xlsb')]

# Load and concatenate all EF_W_ONSHORE_WELLS data with progress tracking
wells_data_list = []
print("Loading EF_W_ONSHORE_WELLS files...")
with tqdm(total=len(all_files)) as pbar:
    for file in all_files:
        wells_data = pd.read_excel(file, usecols=['FACILITY_ID', 'WELL_ID_NUMBER', 'SUB_BASIN'], engine='pyxlsb')

        # Extract the year from the filename (assuming year is in the filename)
        year = ''.join(filter(str.isdigit, os.path.basename(file)))
        wells_data['Year'] = year

        wells_data_list.append(wells_data)
        pbar.update(1)

# Combine all the data into one DataFrame
wells_data = pd.concat(wells_data_list, ignore_index=True)

wells_data = wells_data.dropna(subset=['WELL_ID_NUMBER'])

wells_data['WELL_ID_NUMBER'] = wells_data['WELL_ID_NUMBER'].astype(str)

# Convert scientific notation to full numeric format
scientific_notation_wells = wells_data[
    wells_data['WELL_ID_NUMBER'].str.contains(r'E\+', case=False, na=False)]

# Convert these values to floats, then back to full numbers (string format)
wells_data.loc[scientific_notation_wells.index, 'WELL_ID_NUMBER'] = \
    wells_data.loc[scientific_notation_wells.index, 'WELL_ID_NUMBER'].apply(lambda x: '{:.0f}'.format(float(x)))

# Remove dashes ('-') if present
wells_data['WELL_ID_NUMBER'] = wells_data['WELL_ID_NUMBER'].str.replace('-', '')

# Identify and remove rows where WELL_ID_NUMBER contains letters
# wells_data = wells_data[~wells_data['WELL_ID_NUMBER'].str.contains(r'[A-Za-z]', regex=True, na=False)]
wells_data['WELL_ID_NUMBER'].str.replace(r'\D+', '', regex=True)

# Convert numeric well IDs to int and back to string (removes leading zeros)
wells_data['WELL_ID_NUMBER'] = wells_data['WELL_ID_NUMBER'].str.lstrip('0')  # Remove leading zeros

# Remove well IDs shorter than 9 characters
# wells_data = wells_data[wells_data['WELL_ID_NUMBER'].str.len() >= 9]

# Rename WELL_ID_NUMBER to API14 for merging

wells_data['WELL_ID_NUMBER'] = wells_data['WELL_ID_NUMBER'].apply(
    lambda x: x.ljust(14, '0') if len(x) == 12 else
    x.ljust(13, '0') if len(x) == 11 else
    x.ljust(14, '0') if len(x) == 10 else
    x.ljust(13, '0') if len(x) == 9 else x
)

# Load all Enverus well production data grouped by year with progress tracking
print("Loading well production data...")
all_production_files = [
    os.path.join(wells_grouped_by_year_folder, file)
    for file in os.listdir(wells_grouped_by_year_folder)
]

production_data_list = []
for file in tqdm(all_production_files, desc="Processing well production files"):
    production_data = pd.read_csv(file)
    # Filter to keep only years 2015 and later
    production_data = production_data[production_data['Year'] >= 2015]
    production_data_list.append(production_data)
production_data = pd.concat(production_data_list, ignore_index=True)

production_data['API14'] = production_data['API14'].astype(int)
production_data['API14'] = production_data['API14'].astype(str)
production_data['API14'] = production_data['API14'].str.lstrip('0')  # Remove leading zeros

wells_data['Year'] = wells_data['Year'].astype(int)

# Merge the production data with wells data for the same year
api_facility_correspondance_df = production_data.merge(
    wells_data, 
    left_on=['API14', 'Year'], 
    right_on=['WELL_ID_NUMBER', 'Year'],
    how='inner'  # Ensure we only keep matches
)

wells_ghgrp_prod = api_facility_correspondance_df.merge(ghgrp_production, left_on=['FACILITY_ID', 'Year','SUB_BASIN'], right_on=['facility_id', 'reporting_year','sub_basin_identifier'])

wells_ghgrp_prod = wells_ghgrp_prod.drop(columns={'WELL_ID_NUMBER', 'basin_associated_with_facility', 'facility_name',
       'fractionate_ngl', 'gas_oil_well_ratio', 'gas_prod_cal_year_for_sales',
       'gas_prod_cal_year_from_wells', 'facility_id', 'industry_segment',
       'miles_of_trans_pipe', 'oil_prod_cal_year_for_sales',
       'producing_wells_acquired', 'producing_wells_divested',
       'quantity_gas_added', 'quantity_gas_consumed', 'quantity_gas_delivered',
       'quantity_gas_received', 'quantity_gas_stolen',
       'quantity_gas_transferred', 'quantity_gas_withdrawn',
       'quantity_imported', 'quantity_of_gas_injected',
       'quant_gas_transported_gb', 'quantity_of_gas_withdrawn',
       'quant_hc_liq_recd', 'quant_hc_liq_trans', 'reporting_year',
       'sub_basin_county', 'sub_basin_formation_type', 'sub_basin_identifier',
       'us_state', 'well_producing_end_of_year',
       'well_removed_from_production', 'wells_completed', 'County',
       'State_abbr', 'State'})

wells_ghgrp_prod = wells_ghgrp_prod.rename(columns={'ch4_average_mole_fraction':'C1', 'co2_average_mole_fraction':'CO2'})
wells_ghgrp_prod = wells_ghgrp_prod.drop_duplicates()

wells_ghgrp_prod['C1'] *= 100

wells_ghgrp_prod['CO2'] *= 100

output_filename = 'ghgrp_production_processed.csv'
output_path = os.path.join(GHGRPfolder, output_filename)
wells_ghgrp_prod.to_csv(output_path, index=False)

In [ ]:
ghgrp = pd.read_csv(ghgrp_path)
ghgrp_indsegm_not_na = ghgrp[ghgrp.industry_segment.notna()]
ghgrp_processing = ghgrp_indsegm_not_na[ghgrp_indsegm_not_na.industry_segment.str.contains('Onshore natural gas processing')]
# remove 0 values
ghgrp_processing = ghgrp_processing[ghgrp_processing.ch4_average_mole_fraction > 0].reset_index(drop=True)

ghgrp_processing = ghgrp_processing.merge(facilities_lat_lon, left_on=['facility_id'], right_on=['facility_id'])

geometry = [Point(lon, lat) for lat, lon in zip(ghgrp_processing.latitude, ghgrp_processing.longitude)]

ghgrp_processing_gdf = gpd.GeoDataFrame(ghgrp_processing, geometry=geometry, crs='EPSG:4326')
ghgrp_processing_gdf = gpd.sjoin(ghgrp_processing_gdf.to_crs(26914), us_counties.to_crs(26914), op='within')

ghgrp_processing_gdf[['County', 'State']] = ghgrp_processing_gdf['NAME'].str.extract(r'(.+)\s+\((.+)\)')
ghgrp_processing_gdf = ghgrp_processing_gdf.drop(columns={'index_right'})
ghgrp_processing_gdf = gpd.sjoin(ghgrp_processing_gdf.to_crs(26914), basins_gdf.to_crs(26914), op='within')

capacities = pd.read_csv(root_path + '/ghgrp/processing_capacities.csv',  usecols=['State', 
                                                                                   'County',
                                                                                   'Cap_MMcfd',
                                                                                  'Plant_Flow',
                                                                                  'x',
                                                                                  'y'])
geometry = [Point(lon, lat) for lat, lon in zip(capacities.y, capacities.x)]
capacities_gdf = gpd.GeoDataFrame(capacities, geometry=geometry, crs='EPSG:3857')

import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
from shapely.validation import explain_validity

def nearest(row, other_gdf):
    """
    Find the nearest point in `other_gdf` for the point in the row of `gdf1`.
    
    row: A row from gdf1
    other_gdf: The second GeoDataFrame (gdf2) to find the nearest point.
    
    Returns:
        The geometry and index of the nearest point from `other_gdf`.
    """
    # Validate geometry before processing
    if not row.geometry.is_valid:
        print(row)
        print(f"Invalid geometry found: {explain_validity(row.geometry)}")
        return None, None

    # Use shapely's nearest_points to find the closest point
    try:
        nearest_geom = nearest_points(row.geometry, other_gdf.unary_union)[1]
        # Find the corresponding index of the nearest point in gdf2
        nearest_geom_idx = other_gdf.distance(nearest_geom).idxmin()
        return nearest_geom, nearest_geom_idx
    except Exception as e:
        print(f"Error finding nearest point for geometry: {row.geometry} - {str(e)}")
        return None, None

def get_nearest_distances(gdf1, gdf2):
    """
    Find the nearest point in gdf2 for each point in gdf1, compute the distance,
    and return the corresponding row content from gdf2.
    
    gdf1: GeoDataFrame with point geometries (source points)
    gdf2: GeoDataFrame with point geometries (target points)
    
    Returns:
        A GeoDataFrame with distances and nearest point information from gdf2.
    """
    
    # Ensure both GeoDataFrames are in the same CRS
    if gdf1.crs != gdf2.crs:
        gdf1 = gdf1.to_crs(gdf2.crs)

    # Filter out invalid or empty geometries in gdf2
    gdf2 = gdf2[gdf2.is_valid & ~gdf2.is_empty]

    # Initialize lists to store results
    nearest_geometries = []
    nearest_indices = []
    distances = []
    
    # Find the nearest point in gdf2 for each point in gdf1
    for idx, row in gdf1.iterrows():
        nearest_geom, nearest_geom_idx = nearest(row, gdf2)
        nearest_geometries.append(nearest_geom)
        nearest_indices.append(nearest_geom_idx)
        distances.append(row.geometry.distance(nearest_geom) if nearest_geom else None)

    # Add the nearest distances and geometries to gdf1
    gdf1['distance_to_nearest'] = distances
    gdf1['nearest_geom'] = nearest_geometries

    # Merge the nearest row from gdf2 based on the nearest_indices
    nearest_rows = gdf2.loc[nearest_indices].reset_index(drop=True)
    gdf1 = gdf1.reset_index(drop=True)

    # Merge gdf1 with nearest_rows from gdf2
    gdf1 = gdf1.join(nearest_rows, rsuffix='_gdf2')

    return gdf1

# Example usage
ghgrp_processing_gdf = get_nearest_distances(capacities_gdf, ghgrp_processing_gdf)

ghgrp_processing_gdf = ghgrp_processing_gdf.drop(columns={'NAME','index_right','BASIN_CODE'})

ghgrp_processing_gdf = ghgrp_processing_gdf.rename(columns={'reporting_year':'Year'})

ghgrp_processing_gdf = ghgrp_processing_gdf.drop_duplicates()

ghgrp_processing_gdf = ghgrp_processing_gdf.reset_index(drop=True)

ghgrp_processing_gdf = ghgrp_processing_gdf.rename(columns={'Monthly Gas':'Gas'})

ghgrp_processing_gdf['Count'] = [1 for i in range(len(ghgrp_processing_gdf))]

ghgrp_processing_gdf.to_csv(root_path + '/ghgrp_processing_new_weighted_variable_gdf.csv', index=False)